## Input Parameters

In [1]:
# The default is to include points from all NLCD classes and MGRS grid zones
point_nlcd_keep_list = []
point_mgrs_keep_list = []

# # This would show all NLCD classes for MGRS grid zone 12T
# point_nlcd_keep_list = [31, 41, 42, 43, 52, 71, 81, 82, 90, 95]
# point_mgrs_keep_list = ['12T']

# This would show NLCD 82 points for all MGRS grid zones
point_nlcd_keep_list = [82]
point_mgrs_keep_list = []

# The minimum number of months in the target year
min_month_count = 3

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import os
import pprint
import random

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)

    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)
            
        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[]):
    """"""
    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    print('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])
        
        print('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 1577


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)


            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1382   0.1064   0.0028   0.7428   0.1600   0.7454    12616
        climo_mean   0.1522   0.1171   0.0024   0.6868   0.1938   0.6908    12616
      climo_median   0.1577   0.1126   0.0041   0.7232   0.1733   0.6722    12616
     interp_clim_a   0.1277   0.1002   0.0026   0.7148   0.1769   0.7894    12616
     interp_clim_b   0.1282   0.1004   0.0029   0.7148   0.1771   0.7877    12616
     interp_clim_c   0.1285   0.0982   0.0034   0.7330   0.1666   0.7830    12616
       whit_a_0p50   0.1446   0.1114   0.0024   0.7482   0.1563   0.7219    12616
       whit_a_0p20   0.1397   0.1067   0.0018   0.7848   0.1333   0.7422    12616
       whit_a_0p10   0.1381   0.1051   0.0015   0.8039   0.1213   0.7495    12616
       whit_a_0p05   0.1378   0.1046   0.0012   0.8164   0.1134   0.7522    12616
       whit_a_0p01   0.1385   0.1049   0.0008   0.8289   0.1054   0.7519    12616


## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1359   0.1054  -0.0117   0.7441   0.1639   0.7672    12616
        climo_mean   0.1555   0.1170   0.0026   0.6926   0.2136   0.6922    12616
      climo_median   0.1613   0.1120   0.0077   0.7362   0.1888   0.6750    12616
     interp_clim_a   0.1274   0.0996  -0.0046   0.7184   0.1887   0.8025    12616
     interp_clim_b   0.1274   0.0996  -0.0046   0.7183   0.1888   0.8024    12616
     interp_clim_c   0.1279   0.0969  -0.0020   0.7401   0.1764   0.7959    12616
       whit_a_0p50   0.1442   0.1117  -0.0067   0.7365   0.1741   0.7359    12616
       whit_a_0p20   0.1364   0.1046  -0.0042   0.7813   0.1459   0.7639    12616
       whit_a_0p10   0.1331   0.1015  -0.0032   0.8057   0.1302   0.7758    12616
       whit_a_0p05   0.1316   0.1001  -0.0027   0.8221   0.1195   0.7816    12616
       whit_a_0p01   0.1312   0.0996  -0.0023   0.8387   0.1084   0.7848    12616


## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1433   0.1094   0.0207   0.7242   0.1648   0.6255    12582
        climo_mean   0.1464   0.1141   0.0008   0.5807   0.2198   0.5846    12582
      climo_median   0.1503   0.1092  -0.0023   0.6068   0.2031   0.5655    12582
     interp_clim_a   0.1280   0.0996   0.0107   0.6524   0.1923   0.6862    12582
     interp_clim_b   0.1291   0.1002   0.0114   0.6533   0.1925   0.6804    12582
     interp_clim_c   0.1288   0.0983   0.0092   0.6655   0.1839   0.6804    12582
       whit_a_0p50   0.1487   0.1128   0.0126   0.7316   0.1528   0.6025    12582
       whit_a_0p20   0.1467   0.1113   0.0083   0.7528   0.1374   0.6152    12582
       whit_a_0p10   0.1467   0.1114   0.0064   0.7636   0.1299   0.6179    12582
       whit_a_0p05   0.1476   0.1121   0.0053   0.7707   0.1251   0.6169    12582
       whit_a_0p01   0.1495   0.1137   0.0043   0.7779   0.1203   0.6122    12582


## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1428   0.1080   0.0175   0.6764   0.1723   0.6020    11022
        climo_mean   0.1471   0.1145   0.0035   0.5629   0.2125   0.5618    11022
      climo_median   0.1516   0.1094  -0.0011   0.5828   0.1984   0.5381    11022
     interp_clim_a   0.1292   0.0997   0.0105   0.6196   0.1924   0.6674    11022
     interp_clim_b   0.1315   0.1009   0.0119   0.6203   0.1935   0.6543    11022
     interp_clim_c   0.1302   0.0985   0.0082   0.6296   0.1853   0.6590    11022
       whit_a_0p50   0.1446   0.1091   0.0035   0.6841   0.1546   0.5914    11022
       whit_a_0p20   0.1450   0.1093   0.0016   0.7059   0.1422   0.5947    11022
       whit_a_0p10   0.1465   0.1104   0.0014   0.7170   0.1367   0.5915    11022
       whit_a_0p05   0.1484   0.1117   0.0014   0.7242   0.1334   0.5865    11022
       whit_a_0p01   0.1513   0.1141   0.0017   0.7316   0.1301   0.5775    11022


## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1902   0.1492   0.0281   0.5663   0.2736   0.4199     5431
        climo_mean   0.1611   0.1253  -0.0079   0.5384   0.2533   0.5379     5431
      climo_median   0.1672   0.1205  -0.0121   0.5561   0.2392   0.5081     5431
     interp_clim_a   0.1565   0.1229   0.0101   0.5524   0.2634   0.5649     5431
     interp_clim_b   0.1597   0.1252   0.0121   0.5491   0.2674   0.5473     5431
     interp_clim_c   0.1579   0.1220   0.0080   0.5612   0.2564   0.5562     5431
       whit_a_0p50   0.2013   0.1568   0.0043   0.6066   0.2269   0.3929     5431
       whit_a_0p20   0.2008   0.1562  -0.0029   0.6377   0.2021   0.4089     5431
       whit_a_0p10   0.2018   0.1569  -0.0057   0.6545   0.1898   0.4138     5431
       whit_a_0p05   0.2036   0.1581  -0.0072   0.6660   0.1819   0.4143     5431
       whit_a_0p01   0.2068   0.1606  -0.0085   0.6778   0.1739   0.4110     5431


## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))

    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1694   0.1307   0.0028   0.6549   0.2131   0.6093    12616
        climo_mean   0.1520   0.1165   0.0003   0.6818   0.1942   0.6826    12616
      climo_median   0.1572   0.1121   0.0009   0.7189   0.1722   0.6648    12616
     interp_clim_a   0.1405   0.1098   0.0015   0.6683   0.2037   0.7348    12616
     interp_clim_b   0.1416   0.1104   0.0021   0.6671   0.2050   0.7298    12616
     interp_clim_c   0.1406   0.1077   0.0018   0.6869   0.1927   0.7311    12616
       whit_a_0p50   0.1773   0.1358   0.0018   0.6870   0.1926   0.5855    12616
       whit_a_0p20   0.1745   0.1331   0.0013   0.7257   0.1686   0.6054    12616
       whit_a_0p10   0.1740   0.1325   0.0011   0.7463   0.1558   0.6129    12616
       whit_a_0p05   0.1744   0.1327   0.0009   0.7598   0.1473   0.6157    12616
       whit_a_0p01   0.1760   0.1338   0.0008   0.7736   0.1388   0.6152    12616


## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1760   0.1377  -0.0214   0.6395   0.2255   0.6107    12616
        climo_mean   0.1548   0.1170   0.0014   0.6929   0.2117   0.6931    12616
      climo_median   0.1611   0.1128   0.0065   0.7352   0.1878   0.6738    12616
     interp_clim_a   0.1441   0.1143  -0.0100   0.6662   0.2186   0.7436    12616
     interp_clim_b   0.1442   0.1145  -0.0100   0.6660   0.2188   0.7430    12616
     interp_clim_c   0.1439   0.1116  -0.0075   0.6874   0.2066   0.7393    12616
       whit_a_0p50   0.1833   0.1423  -0.0094   0.6568   0.2257   0.5808    12616
       whit_a_0p20   0.1777   0.1370  -0.0062   0.7012   0.1985   0.6098    12616
       whit_a_0p10   0.1755   0.1347  -0.0051   0.7261   0.1825   0.6229    12616
       whit_a_0p05   0.1748   0.1338  -0.0046   0.7432   0.1713   0.6297    12616
       whit_a_0p01   0.1749   0.1337  -0.0043   0.7609   0.1594   0.6340    12616


## Randomly drop a two month gap during the non-growing season

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1685   0.1291   0.0303   0.6600   0.2087   0.5166    12582
        climo_mean   0.1518   0.1181  -0.0019   0.5619   0.2280   0.5615    12582
      climo_median   0.1575   0.1145  -0.0055   0.5818   0.2140   0.5331    12582
     interp_clim_a   0.1410   0.1096   0.0142   0.6110   0.2183   0.6257    12582
     interp_clim_b   0.1432   0.1110   0.0154   0.6099   0.2201   0.6142    12582
     interp_clim_c   0.1425   0.1090   0.0124   0.6209   0.2114   0.6166    12582
       whit_a_0p50   0.1770   0.1345   0.0133   0.6794   0.1815   0.4852    12582
       whit_a_0p20   0.1772   0.1348   0.0079   0.7041   0.1632   0.4934    12582
       whit_a_0p10   0.1787   0.1360   0.0058   0.7171   0.1542   0.4938    12582
       whit_a_0p05   0.1806   0.1376   0.0047   0.7256   0.1487   0.4912    12582
       whit_a_0p01   0.1840   0.1403   0.0037   0.7339   0.1434   0.4845    12582


## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1959   0.1517   0.0059   0.5858   0.2569   0.4905    12616
        climo_mean   0.1509   0.1157  -0.0001   0.6837   0.1915   0.6866    12616
      climo_median   0.1564   0.1112   0.0005   0.7208   0.1697   0.6674    12616
     interp_clim_a   0.1512   0.1194   0.0029   0.6348   0.2242   0.6896    12616
     interp_clim_b   0.1527   0.1203   0.0037   0.6330   0.2261   0.6831    12616
     interp_clim_c   0.1511   0.1172   0.0032   0.6533   0.2133   0.6877    12616
       whit_a_0p50   0.2078   0.1594   0.0052   0.6372   0.2251   0.4677    12616
       whit_a_0p20   0.2079   0.1590   0.0039   0.6743   0.2013   0.4820    12616
       whit_a_0p10   0.2089   0.1596   0.0032   0.6940   0.1886   0.4873    12616
       whit_a_0p05   0.2102   0.1606   0.0026   0.7070   0.1801   0.4889    12616
       whit_a_0p01   0.2127   0.1626   0.0018   0.7200   0.1715   0.4877    12616


## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2064   0.1599  -0.0292   0.5666   0.2653   0.4820    12616
        climo_mean   0.1515   0.1151   0.0008   0.7054   0.2010   0.7069    12616
      climo_median   0.1570   0.1100   0.0061   0.7451   0.1793   0.6902    12616
     interp_clim_a   0.1551   0.1226  -0.0142   0.6360   0.2332   0.7018    12616
     interp_clim_b   0.1554   0.1228  -0.0141   0.6356   0.2336   0.7006    12616
     interp_clim_c   0.1543   0.1198  -0.0115   0.6559   0.2223   0.7010    12616
       whit_a_0p50   0.2165   0.1663  -0.0101   0.6020   0.2604   0.4525    12616
       whit_a_0p20   0.2141   0.1637  -0.0050   0.6419   0.2384   0.4743    12616
       whit_a_0p10   0.2135   0.1627  -0.0030   0.6639   0.2255   0.4848    12616
       whit_a_0p05   0.2135   0.1626  -0.0019   0.6787   0.2164   0.4905    12616
       whit_a_0p01   0.2144   0.1634  -0.0012   0.6939   0.2068   0.4941    12616


## Randomly drop a three month gap during the non-growing season

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1831   0.1413   0.0410   0.6246   0.2403   0.4690    12582
        climo_mean   0.1505   0.1166  -0.0005   0.5771   0.2241   0.5855    12582
      climo_median   0.1561   0.1127  -0.0038   0.6006   0.2083   0.5578    12582
     interp_clim_a   0.1461   0.1143   0.0202   0.6009   0.2322   0.6176    12582
     interp_clim_b   0.1487   0.1160   0.0218   0.5989   0.2348   0.6042    12582
     interp_clim_c   0.1469   0.1134   0.0186   0.6126   0.2243   0.6116    12582
       whit_a_0p50   0.1950   0.1492   0.0186   0.6556   0.2015   0.4296    12582
       whit_a_0p20   0.1967   0.1502   0.0112   0.6792   0.1816   0.4335    12582
       whit_a_0p10   0.1991   0.1519   0.0081   0.6916   0.1719   0.4320    12582
       whit_a_0p05   0.2018   0.1541   0.0063   0.6997   0.1659   0.4281    12582
       whit_a_0p01   0.2060   0.1577   0.0048   0.7073   0.1602   0.4202    12582


## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2149   0.1661   0.0083   0.5268   0.2940   0.4024    12616
        climo_mean   0.1511   0.1157   0.0030   0.6825   0.1948   0.6852    12616
      climo_median   0.1559   0.1108   0.0042   0.7214   0.1725   0.6693    12616
     interp_clim_a   0.1591   0.1252   0.0056   0.6047   0.2444   0.6554    12616
     interp_clim_b   0.1610   0.1264   0.0068   0.6015   0.2475   0.6463    12616
     interp_clim_c   0.1585   0.1229   0.0062   0.6241   0.2333   0.6556    12616
       whit_a_0p50   0.2302   0.1760   0.0078   0.5885   0.2564   0.3818    12616
       whit_a_0p20   0.2330   0.1775   0.0074   0.6225   0.2354   0.3904    12616
       whit_a_0p10   0.2353   0.1790   0.0071   0.6400   0.2245   0.3928    12616
       whit_a_0p05   0.2376   0.1809   0.0068   0.6513   0.2174   0.3925    12616
       whit_a_0p01   0.2413   0.1837   0.0064   0.6621   0.2105   0.3892    12616


## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2288   0.1766  -0.0363   0.5066   0.2935   0.3914    12616
        climo_mean   0.1519   0.1146   0.0020   0.7072   0.1977   0.7096    12616
      climo_median   0.1568   0.1097   0.0075   0.7470   0.1767   0.6950    12616
     interp_clim_a   0.1647   0.1300  -0.0171   0.6069   0.2456   0.6675    12616
     interp_clim_b   0.1655   0.1305  -0.0168   0.6053   0.2470   0.6640    12616
     interp_clim_c   0.1632   0.1270  -0.0144   0.6268   0.2351   0.6698    12616
       whit_a_0p50   0.2444   0.1859  -0.0140   0.5541   0.2841   0.3579    12616
       whit_a_0p20   0.2476   0.1874  -0.0074   0.5888   0.2674   0.3655    12616
       whit_a_0p10   0.2500   0.1888  -0.0048   0.6075   0.2575   0.3685    12616
       whit_a_0p05   0.2521   0.1903  -0.0035   0.6199   0.2506   0.3695    12616
       whit_a_0p01   0.2551   0.1925  -0.0026   0.6325   0.2430   0.3689    12616


## Randomly drop a four month gap during the non-growing season

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2043   0.1578   0.0519   0.5842   0.2768   0.4064    12582
        climo_mean   0.1528   0.1182   0.0027   0.5864   0.2264   0.5987    12582
      climo_median   0.1579   0.1135  -0.0002   0.6163   0.2074   0.5741    12582
     interp_clim_a   0.1564   0.1223   0.0273   0.5853   0.2516   0.5921    12582
     interp_clim_b   0.1592   0.1241   0.0289   0.5819   0.2551   0.5785    12582
     interp_clim_c   0.1567   0.1210   0.0258   0.6002   0.2421   0.5892    12582
       whit_a_0p50   0.2198   0.1682   0.0289   0.6306   0.2288   0.3690    12582
       whit_a_0p20   0.2222   0.1698   0.0207   0.6551   0.2073   0.3725    12582
       whit_a_0p10   0.2246   0.1716   0.0168   0.6680   0.1964   0.3721    12582
       whit_a_0p05   0.2272   0.1737   0.0145   0.6766   0.1895   0.3698    12582
       whit_a_0p01   0.2314   0.1770   0.0124   0.6850   0.1828   0.3643    12582


## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2435   0.1886   0.0026   0.4432   0.3415   0.2819    12616
        climo_mean   0.1516   0.1163   0.0003   0.6844   0.1924   0.6859    12616
      climo_median   0.1564   0.1115   0.0014   0.7213   0.1710   0.6695    12616
     interp_clim_a   0.1713   0.1349   0.0014   0.5638   0.2669   0.6011    12616
     interp_clim_b   0.1734   0.1363   0.0028   0.5597   0.2708   0.5908    12616
     interp_clim_c   0.1707   0.1328   0.0020   0.5822   0.2563   0.6026    12616
       whit_a_0p50   0.2682   0.2049   0.0025   0.5124   0.2992   0.2604    12616
       whit_a_0p20   0.2772   0.2112   0.0018   0.5428   0.2800   0.2593    12616
       whit_a_0p10   0.2829   0.2154   0.0010   0.5569   0.2707   0.2568    12616
       whit_a_0p05   0.2873   0.2188   0.0002   0.5655   0.2647   0.2539    12616
       whit_a_0p01   0.2928   0.2233  -0.0008   0.5730   0.2591   0.2491    12616


## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2547   0.1976  -0.0358   0.4140   0.3369   0.2688    12616
        climo_mean   0.1496   0.1140   0.0047   0.7114   0.1882   0.7153    12616
      climo_median   0.1554   0.1099   0.0085   0.7472   0.1693   0.6972    12616
     interp_clim_a   0.1738   0.1381  -0.0155   0.5627   0.2626   0.6247    12616
     interp_clim_b   0.1750   0.1390  -0.0148   0.5599   0.2651   0.6183    12616
     interp_clim_c   0.1727   0.1357  -0.0136   0.5806   0.2531   0.6259    12616
       whit_a_0p50   0.2805   0.2145  -0.0234   0.4673   0.3154   0.2348    12616
       whit_a_0p20   0.2894   0.2207  -0.0179   0.4926   0.3047   0.2316    12616
       whit_a_0p10   0.2949   0.2248  -0.0157   0.5051   0.2990   0.2289    12616
       whit_a_0p05   0.2992   0.2283  -0.0146   0.5131   0.2951   0.2263    12616
       whit_a_0p01   0.3045   0.2329  -0.0138   0.5207   0.2910   0.2223    12616


## Randomly drop a six month gap during the non-growing season

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2269   0.1767   0.0470   0.5236   0.3239   0.3375    12582
        climo_mean   0.1513   0.1167  -0.0007   0.6420   0.2074   0.6446    12582
      climo_median   0.1561   0.1116  -0.0020   0.6800   0.1840   0.6263    12582
     interp_clim_a   0.1645   0.1290   0.0232   0.5828   0.2657   0.5882    12582
     interp_clim_b   0.1672   0.1308   0.0249   0.5787   0.2697   0.5756    12582
     interp_clim_c   0.1641   0.1271   0.0225   0.6018   0.2540   0.5901    12582
       whit_a_0p50   0.2514   0.1939   0.0356   0.6017   0.2671   0.3108    12582
       whit_a_0p20   0.2600   0.2003   0.0297   0.6315   0.2439   0.3070    12582
       whit_a_0p10   0.2654   0.2045   0.0266   0.6449   0.2329   0.3030    12582
       whit_a_0p05   0.2697   0.2080   0.0243   0.6526   0.2263   0.2988    12582
       whit_a_0p01   0.2752   0.2126   0.0219   0.6587   0.2202   0.2921    12582


### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )